In [ ]:
import torch 
import transformers
from dataclasses import dataclass, field 
from typing import Dict, Optional, Sequence 
from trainer_seq2seq import Seq2SeqTrainer
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainingArguments,
    set_seed,
)

model_name_or_path = r"/root/autodl-tmp/chatglm2-6b-int4"
cache_dir = r"/root/autodl-tmp/chatglm2-6b-int4"

In [ ]:
import json

data_train = []
with open("/root/autodl-tmp/fix-helper/LLM-langchain/Generate-from-openai/generate_file2.json",'r',encoding="utf-8") as file:
    data_train = json.loads(file.read())

In [ ]:
@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""
    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self):
        

In [ ]:
def train(
    model_name_or_path,
    cache_dir,
    model_max_length,
    output_dir,
    learning_rate = 5e-5,
    warmup_ratio=0.8,
    num_train_epochs = 3,
):
    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name_or_path,
        cache_dir=cache_dir,
    )
    tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_name_or_path,
        cache_dir=cache_dir,
        model_max_length=model_max_length,
        padding_side="right",
        use_fast=False,
    )
    training_args = transformers.TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir = True,
        evaluation_strategy = "steps",
        learning_rate = learning_rate,
        warmup_ratio = warmup_ratio,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=4,
        num_train_epochs=num_train_epochs,
        save_steps=False,
    )
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset if do_train else None,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics if predict_with_generate else None,
        save_changed=model_args.pre_seq_len is not None
    )